In [12]:
import pickle
import pandas as pd
import torch
import io

from run_resnet import LayerwiseQuantizationProblem, QuantizationModel, CalibrationModel

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)


with open('exploration.pkl', 'rb') as f:
    d = CPU_Unpickler(f).load()

type(d)

pymoo.core.result.Result

In [17]:
from torchvision import models

m = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
layer_names = []

for name, module in m.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        layer_names.append(name)


In [29]:
d.opt[0].get("F")

array([ -0.7586, 440.    ])

In [34]:
import numpy as np

pop = d.pop

df_data = np.empty( (55, ) )
for ind in d.opt:
    #l = ind.get("F") + ind.get("X")
    
    print(type(ind.get("x")))

df = pd.pandas(df_data, columns=["Acc", "Bits"] + layer_names)

df
# print(*zip(pop.get("X")[2], layer_names))

TypeError: array() missing required argument 'object' (pos 0)